### This part of the notebook is to achieve the following:
- Build a machine learning model from the combined data
- Make predicitons based on user defined input 
- translate predictions into "Football" languages and return to user 

### Import Modules

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import bz2
import _pickle as cPickle
import dill

from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, RandomizedSearchCV, learning_curve
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, RocCurveDisplay

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier


from sklearn.model_selection import RandomizedSearchCV
from bayes_opt import BayesianOptimization
from sklearn.pipeline import Pipeline
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score, RocCurveDisplay



from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.base import BaseEstimator, RegressorMixin, TransformerMixin
from sklearn.pipeline import Pipeline
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_validate

### load preprocessed the data from compressed pickle

In [4]:
#with bz2.BZ2File("NFL_web_app/model_data/20220428_final_combined_NFL_data.pbz2", "rb") as f:

    final_combined_data = cPickle.load(f)
    
    
print(f"The preprocessed data shape is {final_combined_data.shape}, header is \n")

final_combined_data.head().T

The preprocessed data shape is (114827, 1434), header is 



,0,1,2,3,4
Offense_0,82.0,82.0,82.0,82.0,82.0
Offense_1,80.0,80.0,80.0,80.0,80.0
Offense_2,87.0,87.0,87.0,87.0,87.0
Offense_3,76.0,76.0,76.0,76.0,76.0
Offense_4,81.0,81.0,81.0,81.0,81.0
Offense_5,80.0,80.0,80.0,80.0,80.0
Offense_6,86.0,86.0,86.0,86.0,86.0
Offense_7,69.0,69.0,69.0,69.0,69.0
Offense_8,29.0,29.0,29.0,29.0,29.0
Offense_9,22.0,22.0,22.0,22.0,22.0


### Generate train and test data with an 80/20 split

In [3]:
#split train and test data

#X = final_res.drop(columns=["SeriesFirstDown", "GameId", 'level_1', "GameDate"])
#y = final_res.SeriesFirstDown.tolist()

X = final_combined_data.drop(columns=["Yards_vs_ToGo"], axis=1)

y = final_combined_data.Yards_vs_ToGo.tolist()

split_ratio = 0.2

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=split_ratio, random_state=42)


print(f"X_train shape is {X_train.shape}")
print(f"X_test shape is {X_test.shape}")

print(f"y_train shape is {pd.DataFrame(y_train).shape}")
print(f"y_test shape is {pd.DataFrame(y_test).shape}")

X_train shape is (91861, 1433)
X_test shape is (22966, 1433)
y_train shape is (91861, 1)
y_test shape is (22966, 1)


### Remove unwanted columns in both train and test data
- Drop columns: "SeriesFirstDown", "GameId", "GameDate", "ToGo", "Yards", "Description", "SeasonYear", 
- Formation, OffenseTeam, DefenseTeam, Quarter, Minute, Second

In [6]:
X_train_final = X_train.drop(columns=["SeriesFirstDown", "GameId", "GameDate", 
                                     "ToGo", "Yards", "Description", "SeasonYear",
                                      "Formation", "OffenseTeam", "DefenseTeam", "Quarter", "Minute", "Second"
                                     ], axis=1)

X_test_final = X_test.drop(columns=["SeriesFirstDown", "GameId", "GameDate", 
                                     "ToGo", "Yards", "Description", "SeasonYear",
                                      "Formation", "OffenseTeam", "DefenseTeam", "Quarter", "Minute", "Second"
                                     ], axis=1)

print(f"X_train shape is {X_train_final.shape}")
print(f"X_test shape is {X_test_final.shape}")

print(f"y_train shape is {pd.DataFrame(y_train).shape}")
print(f"y_test shape is {pd.DataFrame(y_test).shape}")


X_train shape is (91861, 1420)
X_test shape is (22966, 1420)
y_train shape is (91861, 1)
y_test shape is (22966, 1)


### Fit the data with linear plus nonlinear regression model
- First fit the data with regularized linear model (Ridge)
- Then fit the residual with random forest regression model
- Combine linear and non-linear fits to genearte the final prediction results

In [3]:
class linear_plus_nonlinear(BaseEstimator, TransformerMixin):
    
    '''
     Define the linear + non-linear ensemble predictor model class
    '''
    def __init__(self, alpha=1, max_depth=1, min_samples_leaf=1, n_estimators=100):
        
        self = self
        self.alpha = alpha
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.n_estimators = n_estimators
        
    
    def fit(self, X, y):
        ridge = Ridge(alpha=self.alpha)
        rf = RandomForestRegressor(max_depth=self.max_depth, min_samples_leaf=self.min_samples_leaf, n_estimators=self.n_estimators)
        self.linear_fit = ridge.fit(X, y)
        residual = y - (self.linear_fit.predict(X))
        #print(f"residual shape is {residual.shape}")
        self.nonlinear_fit = rf.fit(X, residual)
        
        return self
    
    def predict(self, X):
        return self.linear_fit.predict(X) + self.nonlinear_fit.predict(X)
    
    
    '''def score(self, X, y):
        predict_y = self.predict(X)
        return 1 - np.dot(np.array(predict_y - y).T, np.array(predict_y -y)) / np.dot(np.array(y-np.mean(y)).T, np.array(y-np.mean(y)))
    '''
    


In [ ]:
def my_eval(n_estimators, min_samples_leaf, max_depth, alpha):
    
    '''
    Searching for best hyperparameters for the model
    using Bayesian Optimizer
    '''
    model = Pipeline([('scaler', StandardScaler()),
                      ('mix', linear_plus_nonlinear(alpha=alpha, min_samples_leaf=int(min_samples_leaf),
                                                       max_depth=int(max_depth), n_estimators=int(n_estimators)))]) 
    
    cv_score = cross_validate(model, X_train_final, y_train, cv=5, scoring='r2', n_jobs=-1)
    
    return np.mean(cv_score['test_score'])



param_grid = param_grid = {'alpha': (1e-4, 1e4), 'max_depth': (2, 50), 'min_samples_leaf': (2, 100), 'n_estimators': (10, 500)}

myBO = BayesianOptimization(my_eval, param_grid, verbose = 2,
                                 random_state = 42)

myBO.maximize(n_iter=100, init_points=2)

|   iter    |  target   |   alpha   | max_depth | min_sa... | n_esti... |
-------------------------------------------------------------------------
|  1        |  0.08442  |  3.745e+0 |  47.63    |  73.74    |  303.3    |
|  2        |  0.08104  |  1.56e+03 |  9.488    |  7.692    |  434.4    |
|  3        |  0.07645  |  7.231e+0 |  34.07    |  23.84    |  41.31    |
|  4        |  0.08396  |  4.631e+0 |  30.18    |  58.53    |  489.3    |
|  5        |  0.05708  |  9.999e+0 |  2.06     |  30.28    |  490.8    |
|  6        |  0.08473  |  9.708e+0 |  36.06    |  99.4     |  193.2    |
|  7        |  0.08317  |  4.63e+03 |  30.58    |  45.73    |  480.1    |
|  8        |  0.07554  |  9.527e+0 |  46.95    |  50.13    |  12.93    |
|  9        |  0.08456  |  4.854e+0 |  20.29    |  99.67    |  484.5    |
|  10       |  0.08458  |  3.988e+0 |  32.39    |  83.81    |  412.4    |


### Test the best model (mixed_best)

In [14]:
mixed_best = linear_plus_nonlinear(alpha=9.708e+0, min_samples_leaf=int(99.4),
                                                       max_depth=int(36.06), n_estimators=int(193.2))
print(mixed_best)

linear_plus_nonlinear(alpha=9.708, max_depth=36, min_samples_leaf=99,
                      n_estimators=193)


In [8]:
#mixed_best.fit(X_train_final, y_train)

y_pred = mixed_best.predict(X_test_final)

In [9]:
r_square = np.dot((np.array(y_pred) - np.mean(y_test)).T, (np.array(y_pred) - np.mean(y_test))) / np.dot((np.array(y_test) - np.mean(y_test)).T, (np.array(y_test) - np.mean(y_test)))

In [10]:
r_square

0.08483766429343653

In [11]:
X_test["Yards_vs_ToGo"] = y_test
X_test["Predicted_Yards_toGO"] = y_pred

In [15]:
X_test["Predicted_Yards_toGO"].describe()

count    22966.000000
mean         1.054626
std          0.673303
min         -0.147623
25%          0.570218
50%          0.877649
75%          1.383015
max          7.332787
Name: Predicted_Yards_toGO, dtype: float64

### Local test of the web app prototype
- Create a self-defined mock user input for season year, offense/defense team, quarter, yards, down, etc.
- Enumerate all possible pass/rush directions and formations, transform into one hot matrix and combine together with user-defined input features.
- Make predictions on the combined dataset, using a self-defined sigmoid function to convert the response variable to probabilities.
- Rank in descending order every combination of pass/rush directions according to calculated sigmoid probabilities, defined as "chance of success", select Top 5, tranlate them into "Football" language and output to the user. 

In [6]:

'''
Self define a mock user input
'''

#input criteria:
year = 2021
offenseTeam = "MIA"
defenseTeam = "NYJ"

Quarter = 4
minutes = 10
seconds = 5

Down = 1

Yardline = 98

togo = 2


array(['ARI', 'ATL', 'BAL', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'DAL',
       'DEN', 'DET', 'GB', 'HOU', 'IND', 'JAX', 'KC', 'LA', 'LAC', 'LV',
       'MIA', 'MIN', 'NE', 'NO', 'NYG', 'NYJ', 'PHI', 'PIT', 'SEA', 'SF',
       'TB', 'TEN', 'WAS'], dtype=object)

In [5]:
'''
Augment data based on combination of PassType and Rushdirection for each Formation

Note that NOT all teams have games against each other in a season, so the following script may result in bugs:

subset_combined = final_combined_data.loc[np.logical_and(np.logical_and(final_combined_data.OffenseTeam=="MIA", \
                                                         final_combined_data.DefenseTeam=="NYJ"), \
                                                         final_combined_data.SeasonYear==2018)]         

'''



def list_all_PassType_Rushdirection(formation_df, final_Pass_Rush_combined):
    
    '''
    Function to create all combination of PassType and Rushdirection for each Formation
    '''
    
    return_df = pd.concat([formation_df.iloc[0,:]]*final_Pass_Rush_combined.shape[0], axis=1, ignore_index=True).T
    
    return_df.loc[:,final_Pass_Rush_combined.columns.tolist()] = final_Pass_Rush_combined
    
    
    return(return_df)
    
    


#step 1 get all the data for a given offense team in a season
offense_data = final_combined_data.loc[np.logical_and(final_combined_data.OffenseTeam=="MIA", \
                                                         final_combined_data.SeasonYear==2018)].reset_index(drop=True)
#step 2 get all the data for a given offense team in a season
defense_data = final_combined_data.loc[np.logical_and(final_combined_data.DefenseTeam=="LA", \
                                                      final_combined_data.SeasonYear==2018)].reset_index(drop=True)


#step 3 replace all the defence variables in the offensedata with the columns in the DefenseTeam


offense_data.loc[:, [cname for cname in offense_data.columns if "Defense" in cname]] = \
pd.concat([defense_data.loc[0, [cname for cname in defense_data.columns if "Defense" in cname]]]\
          *offense_data.shape[0], axis=1, ignore_index=True).T


subset_combined = offense_data
                                                      
#step 4 remove offense and defense data                                                 

del(offense_data)

del(defense_data)




#step 5 get list of all unique formation from the data, then augment each formation with 
#all possible combinations of PassType and RushDirections

subset_combined = subset_combined.reset_index(drop=True)


list_of_unique_formations = subset_combined.Formation.tolist()



list_of_unique_PassType = [x.split("_")[1] for x in subset_combined.columns if "PassType" in x]

list_of_unique_RushDirection = [x.split("_")[1] for x in subset_combined.columns if "RushDirection" in x]



PassType_dummies = pd.get_dummies(list_of_unique_PassType, prefix='PassType')

RushDirection_dummies = pd.get_dummies(list_of_unique_RushDirection , prefix='RushDirection')

empty_RushDirection = pd.DataFrame(np.zeros(shape=(PassType_dummies.shape[0], len(list_of_unique_RushDirection))),
                                  columns=RushDirection_dummies.columns)
empty_PassType = pd.DataFrame(np.zeros(shape=(RushDirection_dummies.shape[0], len(list_of_unique_PassType))),
                                  columns=PassType_dummies.columns)



final_PassType_df = pd.concat([PassType_dummies, empty_RushDirection], axis=1)

final_PassType_df['PlayType_PASS'] = 1
final_PassType_df['PlayType_RUSH'] = 0


final_RushDirection_df = pd.concat([empty_PassType, RushDirection_dummies], axis=1)

final_RushDirection_df ['PlayType_PASS'] = 0
final_RushDirection_df ['PlayType_RUSH'] = 1


final_Pass_Rush_combined = pd.concat([final_PassType_df, final_RushDirection_df], axis=0, ignore_index=True)



#Step 6. Combined both datasets to form the final "test data"

subset_combined = subset_combined.groupby(['Formation']).\
apply(lambda x: list_all_PassType_Rushdirection(x, final_Pass_Rush_combined))

print(subset_combined.shape)




(92, 1434)


### Final processing of the test data
- Assign user defined values to corresponding columns
- Drop unwanted columns
- self define sigmoid function to convert response variable to probabilities

In [8]:

subset_combined["Time_in_seconds"] = Quarter*15*60 - minutes*60 - seconds 

subset_combined["YardLine"] = Yardline


subset_combined["Down"] = Down

In [24]:
all_combination_test = subset_combined.drop(columns=["SeriesFirstDown", "GameId", "GameDate", 
                                     "ToGo", "Yards", "Description", "SeasonYear",
                                      "Formation", "OffenseTeam", "DefenseTeam", "Quarter", "Minute", "Second", "Yards_vs_ToGo"
                                     ], axis=1)


all_combination_test.shape

(92, 1420)

In [25]:
def sigmoid(alpha, x):
    '''
    Define sigmoid function to convert y pred value to probabilities
    '''
    return 1.0 / (1.0 + np.exp(0-alpha*x))


y_pred = mixed_best.predict(all_combination_test)


sigmoid_y_pred = [sigmoid((max(100-Yardline-togo, 0)+50)/100, x - Down/2) for x in y_pred]


all_combination_test["sigmoid_y_pred"] = sigmoid_y_pred


final_res_df = all_combination_test.sort_values(["sigmoid_y_pred"], ascending=False).head()

print(final_res_df)



                    Offense_0 Offense_1 Offense_2 Offense_3 Offense_4  \
Formation                                                               
NO HUDDLE SHOTGUN 3      77.0      82.0      86.0      67.0      83.0   
SHOTGUN           3      77.0      82.0      86.0      67.0      83.0   
UNDER CENTER      3      77.0      82.0      86.0      67.0      83.0   
NO HUDDLE         3      77.0      82.0      86.0      67.0      83.0   
NO HUDDLE SHOTGUN 2      77.0      82.0      86.0      67.0      83.0   

                    Offense_5 Offense_6 Offense_7 Offense_8 Offense_9  ...  \
Formation                                                              ...   
NO HUDDLE SHOTGUN 3      59.0      81.0      73.0      63.0      26.0  ...   
SHOTGUN           3      59.0      81.0      73.0      63.0      26.0  ...   
UNDER CENTER      3      59.0      81.0      73.0      63.0      26.0  ...   
NO HUDDLE         3      59.0      81.0      73.0      63.0      26.0  ...   
NO HUDDLE SHOTGUN 2 

### Translate results into football language with "chance of success" (as sigmoid y pred)

In [22]:
def python_to_football(res_row):
    
    '''
    Function to translate each record into "Football" languages
    '''
    
    formation = ""
    playType = ""
    passType = ""
    rushDirection = ""
    successRate = ""
    
    list_columns = final_res_df.columns.tolist()
    
    assert len(res_row) == len(list_columns)
    
    for i in range(len(res_row)):
        
        if "Formation_" in list_columns[i] and res_row[i] == 1:
            formation = list_columns[i].split("_")[1]
        
        if "PlayType_PASS" in list_columns[i] and res_row[i] == 1:
            playType = "PASS"
        elif "PlayType_RUSH" in list_columns[i] and res_row[i] == 1:
            playType = "RUSH"
        
        if "PassType_" in list_columns[i] and res_row[i] == 1:
            passType = list_columns[i].split("_")[1]
            
        if "RushDirection_" in list_columns[i] and res_row[i] == 1:
            rushDirection = list_columns[i].split("_")[1]
        
        if "sigmoid_" in list_columns[i]:
            successRate = f"{round(res_row[i]*100, 2)}%"
            
            
        
            
    
    if passType != "":
        return(f"{formation} Formation, {playType} Play, {passType}, with {successRate} chance of Success.")
    elif rushDirection != "":
        return(f"{formation} Formation, {playType} Play, {rushDirection}, with {successRate} chance of Success.")



final_res_df.apply(lambda x: python_to_football(x), axis=1).reset_index(drop=True).to_json()



'{"0":"NO HUDDLE SHOTGUN Formation, PASS Play, DEEP MIDDLE, with 65.31% chance of Success.","1":"SHOTGUN Formation, PASS Play, DEEP MIDDLE, with 63.88% chance of Success.","2":"UNDER CENTER Formation, PASS Play, DEEP MIDDLE, with 63.88% chance of Success.","3":"NO HUDDLE Formation, PASS Play, DEEP MIDDLE, with 63.28% chance of Success.","4":"NO HUDDLE SHOTGUN Formation, PASS Play, DEEP LEFT, with 62.08% chance of Success."}'

### Save the trained model into pickle file with dill to avoid potential bugs when deploying on Heroku

In [53]:
#with open("050122_linear_non_linear_model_trained_dill.pkl", "wb") as f:
    dill.dump(mixed_best, f, recurse=True)
    
print("Finished saving the trained model!")

Finished saving the trained model!


### Save only (absolutely) neccessary part of the data for predictions in order to reduce memory usage in heroku

In [ ]:
'''
only keep records with unique offense team, formation, and season year for test data
'''
final_combined_data = final_combined_data.drop_duplicates(subset=["OffenseTeam", "DefenseTeam", "Formation", "SeasonYear"])

In [1]:
final_combined_data.shape

(5332, 1434)


In [ ]:
'''
Confirm for each season year, there are 32 unique offense and 32 unique defense teams after filtering
'''

assert len(final_combined_data.loc[final_combined_data.SeasonYear==2018, ["DefenseTeam"]].drop_duplicates()) == 32
assert len(final_combined_data.loc[final_combined_data.SeasonYear==2019, ["DefenseTeam"]].drop_duplicates()) == 32
assert len(final_combined_data.loc[final_combined_data.SeasonYear==2020, ["DefenseTeam"]].drop_duplicates()) == 32
assert len(final_combined_data.loc[final_combined_data.SeasonYear==2021, ["DefenseTeam"]].drop_duplicates()) == 32


assert len(final_combined_data.loc[final_combined_data.SeasonYear==2018, ["OffenseTeam"]].drop_duplicates()) == 32
assert len(final_combined_data.loc[final_combined_data.SeasonYear==2019, ["OffenseTeam"]].drop_duplicates()) == 32
assert len(final_combined_data.loc[final_combined_data.SeasonYear==2020, ["OffenseTeam"]].drop_duplicates()) == 32
assert len(final_combined_data.loc[final_combined_data.SeasonYear==2021, ["OffenseTeam"]].drop_duplicates()) == 32

In [2]:
#with open("060922_filtered_final_combined_data_for_prediction.pkl", 'wb') as f:
    '''
    This is the data that will be used for prediction
    '''
    pickle.dump(final_combined_data, f)
print("Finished saving filtered combined data!")

Finished saving filtered combined data!


### Potential Caveat: Mislabeled record in play-by-play data (see details below) 

In [130]:
#print 10 wronlgy classified records
#Looks very suspicious, need to go back to check specific game details

"""
Conclusion on misclassified data: most likely mislabeled:
for example, in the test data [0], index in full data 34760, gameID 2020112601
The penalty on one second down was not labeled, the 3 down were missing labeled as Punt 
(but actually it was a pass play then QB was sacked)
The 4th down were mislabeled as PASS play but in fact it is a scored field goal

Proof: youtube game video: https://www.youtube.com/watch?v=ZpJ61jmIi60 @7:00m
ESPN: play by play: https://www.espn.com/nfl/playbyplay?gameid=401220244


Temporarily give up on fixing mislabeled data (040322 WH)

"""

wrong_classfied_ids = [i for i in range(len(y_test)) if y_test[i]!=y_pred_xgb[i]]

for i in wrong_classfied_ids[:10]:
    
    print(f"Here is 10 wrongly classified record {i+1}\n")
    
    data_translation(X_test, y_test, y_pred_xgb_prob, i)

    print("\n")


Here is 10 wrongly classified record 1

Label is 0, predicted probability is 0.9248878359794617
{'Quarter': 3, 'Time Left': '13m 23s', 'YardLine': 'DAL 22', 'Offense_team': 'WAS', 'Defense_team': 'DAL', 'number of plays': 2.0, 'yards gain': 14, 'Formation': [{'Formation_NO HUDDLE SHOTGUN': 1.0}, {'Formation_SHOTGUN': 1.0}], 'PlayType': [{'PlayType_PASS': 1.0}, {'PlayType_RUSH': 1.0}], 'PassType': [{'PassType_SHORT LEFT': 1.0}], 'RushDirection': [{'RushDirection_LEFT END': 1.0}]}


Here is 10 wrongly classified record 8

Label is 1, predicted probability is 0.15021052956581116
{'Quarter': 3, 'Time Left': '6m 55s', 'YardLine': 'NO 22', 'Offense_team': 'TB', 'Defense_team': 'NO', 'number of plays': 2.0, 'yards gain': 8, 'Formation': [{'Formation_SHOTGUN': 1.0}, {'Formation_UNDER CENTER': 1.0}], 'PlayType': [{'PlayType_RUSH': 2.0}], 'PassType': [], 'RushDirection': [{'RushDirection_LEFT END': 1.0}, {'RushDirection_RIGHT GUARD': 1.0}]}


Here is 10 wrongly classified record 33

Label is 0, 